In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/ING3/NLP/Projet_NLP_chatbot/Projet_VerFinal/dataset'

[Errno 2] No such file or directory: '/content/drive/MyDrive/ING3/NLP/Projet_NLP_chatbot/Projet_VerFinal/dataset'
/content


## Imports


In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/NLP/Projet_NLP_chatbot/Projet_VerFinal/dataset/wiki_movie_plots_tokenized.csv')

In [ ]:
dataset.head()

,Unnamed: 0,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Plot_tokenized
0,0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...","['bartender', 'work', 'saloon', 'serve', 'drin..."
1,1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","['moon', 'paint', 'smile', 'face', 'hang', 'pa..."
2,2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","['film', 'minute', 'long', 'compose', 'two', '..."
3,3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"['last', '61', 'second', 'consist', 'two', 'sh..."
4,4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,"['early', 'know', 'adaptation', 'classic', 'fa..."


# Exemple d'utilisation avec word2vec

In [ ]:
!pip install gensim

## Entraînement de word2vec sur notre dataset

In [ ]:
from gensim.models import Word2Vec

In [ ]:
plot_tokenized =  dataset['Plot_tokenized']

In [ ]:
# Créer et entraîner le modèle Word2Vec
model = Word2Vec(plot_tokenized, vector_size=100, window=5, sg=0, negative=5, min_count=1, workers=4)

## sauvegarder le modèle

In [ ]:
model.save("_deuxieme_modele_word2vec_wikipedia_movie_plots.model")

#à faire si on relance la session

In [ ]:
from gensim.models import Word2Vec

# Charger le modèle Word2Vec depuis le fichier sauvegardé
model = Word2Vec.load("/content/drive/MyDrive/NLP/Projet_NLP_chatbot/Wikipedia_movie_plots/_deuxieme_modele_word2vec_wikipedia_movie_plots.model")

#Calcul de similarité entre deux plot tokenisé

In [ ]:
def Model_similarity(myModel, tokens1, tokens2):
    # Assure-toi que les deux listes de tokens ne sont pas vides
    if tokens1 and tokens2:
        # Concatène les listes de tokens en deux chaînes de texte
        text1 = ' '.join(tokens1)
        text2 = ' '.join(tokens2)

        # Calcule la similarité entre les deux chaînes de texte
        similarity_score = myModel.wv.wmdistance(text1, text2)

        return similarity_score
    else:
        return None


#test sur les films de  genre western (idée d'exemple)

In [ ]:
western_movies = dataset[dataset['Genre'] == 'western']

In [ ]:
!pip install pot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.0/790.0 kB 15.4 MB/s eta 0:00:00


In [ ]:
tokenized_plot_film1 = western_movies.loc[97,'Plot_tokenized']
tokenized_plot_film2 = western_movies.loc[99,'Plot_tokenized']
Model_similarity(model,tokenized_plot_film1,tokenized_plot_film2)

/usr/local/lib/python3.10/dist-packages/ot/backend.py:1368: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  jax.device_put(jnp.array(1, dtype=jnp.float64), d)
/usr/local/lib/python3.10/dist-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


0.054043126626237704

#ideé suggéré : créer une description utilisateur pour tester le modèle dans le but de calculer la similitude entre descritption et un film (ou plusieurs film)

In [ ]:
tokenized_description = ['adventurous', 'prospector', 'rushes', 'California', 'brave', 'woman', 'Sally', 'Jane', 'rescue', 'rogue', 'Red', 'Pete', 'valiant', 'stranger', 'John', 'Dart', 'latter', 'rescue', 'innocent', 'accused', 'crime']

In [ ]:
#on appelle la fonction définie plus haut
Model_similarity(model,tokenized_description,tokenized_plot_film2)

0.6837276748088692

In [ ]:
western_movies_2008 = western_movies[(western_movies['Release Year'] >= 2008) & (western_movies['Release Year'] <= 2020)]

In [ ]:
western_movies_2008.head()

,Unnamed: 0,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Plot_tokenized
15387,15387,2008,Appaloosa,American,Ed Harris,"Ed Harris, Viggo Mortensen, Renée Zellweger, J...",western,https://en.wikipedia.org/wiki/Appaloosa_(film),"In 1882, the small town of Appaloosa, New Mexi...","['1882', 'small', 'town', 'appaloosa', 'new', ..."
15950,15950,2010,True Grit,American,Joel and Ethan Coen,"Jeff Bridges, Matt Damon, Josh Brolin, Barry P...",western,https://en.wikipedia.org/wiki/True_Grit_(2010_...,Mattie Ross's father was murdered by Tom Chane...,"['mattie', 'rosss', 'father', 'murder', 'tom',..."
16253,16253,2012,Django Unchained,American,Quentin Tarantino,"Jamie Foxx, Christoph Waltz, Leonardo DiCaprio...",western,https://en.wikipedia.org/wiki/Django_Unchained,"In 1858 Texas, the Speck brothers, Ace and Dic...","['1858', 'texas', 'speck', 'brothers', 'ace', ..."
16424,16424,2013,Ain't Them Bodies Saints,American,David Lowery,"Casey Affleck, Rooney Mara, Ben Foster, Rami M...",western,https://en.wikipedia.org/wiki/Ain%27t_Them_Bod...,Ruth Guthrie is walking across a field and is ...,"['ruth', 'guthrie', 'walk', 'across', 'field',..."
17373,17373,2017,Hostiles,American,Scott Cooper,Scott Cooper (director/screenplay); Donald E. ...,western,https://en.wikipedia.org/wiki/Hostiles_(film),"In 1892, settler Wesley Quaid and his family a...","['1892', 'settler', 'wesley', 'quaid', 'family..."


#fonction pour calculer les top k movies à partir d'une référence de film (dans le dataset)

In [ ]:
def get_top_k_similar_movies_from_reference(myModel, dataset, reference_line, k=5):
    reference_tokenized = dataset.loc[reference_line, 'Plot_tokenized']

    # Liste pour stocker les informations des k films les plus similaires
    top_k_movies_info = []

    for line in dataset.index:
        if line != reference_line:
            tokenized_plot = dataset.loc[line, 'Plot_tokenized']
            similarity_score = Model_similarity(myModel, reference_tokenized, tokenized_plot)

            movie_info = {
                'Title': dataset.loc[line, 'Title'],
                'Director': dataset.loc[line, 'Director'],
                'Release Year': dataset.loc[line, 'Release Year'],
                'Similarity Score': similarity_score
            }

            top_k_movies_info.append(movie_info)

    # Trier la liste par scores de similarité décroissants
    top_k_movies_info = sorted(top_k_movies_info, key=lambda x: x['Similarity Score'], reverse=True)[:k]

    return top_k_movies_info


## exemple d'utilisation

In [ ]:
reference_movie_id = 16253 # FIlm DJANGO UNCHAINED
top_k_similar_movies = get_top_k_similar_movies_from_reference(model, western_movies, reference_movie_id, k=3)
print(top_k_similar_movies)

[{'Title': 'The High Powered Rifle', 'Director': 'Maury Dexter', 'Release Year': 1960, 'Similarity Score': 0.1761742432123328}, {'Title': 'The Fargo Kid', 'Director': 'Edward Killy', 'Release Year': 1940, 'Similarity Score': 0.15498128101083553}, {'Title': "Hell's Crossroads", 'Director': 'Franklin Adreon', 'Release Year': 1957, 'Similarity Score': 0.15236376705837032}]


## autre exemple d'utilisation avec les films Harry Potter

In [ ]:
films_harry_potter = dataset[dataset['Title'].str.contains('harry potter', case=False)]

In [ ]:
reference_movie_id = 13862 # Harry Potter
top_k_similar_movies = get_top_k_similar_movies_from_reference(model, films_harry_potter, reference_movie_id, k=3)
print(top_k_similar_movies)

[{'Title': 'Harry Potter and the Goblet of Fire', 'Director': 'Mike Newell', 'Release Year': 2005, 'Similarity Score': 0.030756907850690345}, {'Title': 'Harry Potter and the Order of the Phoenix', 'Director': 'David Yates', 'Release Year': 2007, 'Similarity Score': 0.03034226340695543}, {'Title': 'Harry Potter and the Prisoner of Azkaban', 'Director': 'Alfonso Cuarón', 'Release Year': 2004, 'Similarity Score': 0.028830687098159575}]


#Conclusion:
On observe que les coeficients de similarités obtenues avec Word2Vec sont très faibles,c'est la raison pour laquelle on a choisi de travailler les calculs de similarités avec BERT